In [1]:
%%javascript
MathJax.Hub.Config({TeX: { equationNumbers: { autoNumber: "AMS" } }});

<IPython.core.display.Javascript object>

# E-BMFLC - Enhanced Band-limited Multiple Fourier Linear Combiner 



The Enhanced Band-limited Multiple Fourier Linear Combiner  (E-BMFLC) filter is an improvement on the BMWFLC. The E-BMFLC filter was introduced in this [paper](https://ieeexplore.ieee.org/document/7407298), and a new dynamic estimation strategy for the E-BMFLC in this [paper](https://ieeexplore.ieee.org/document/8308681), but we will not review the latter one at this time.  

The following model represents the hand movement of a patient with pathological hand tremor

\begin{equation}\label{eq:motion}
m_k = m_{k-v}+m_{k-i}
\end{equation}

In \eqref{eq:motion}, $m_k$ is the signal of the patients hand motion; $m_{k-v}$ and $m_{k-i}$ are the voluntary and involuntary components of the motion respectively. The main goal of the BMFLC was to estimate the involuntary motion $m_{k-i}$  in real-time. The E-BMFLC have been developed to address two two major drawbacks of the BMFLC filter, inaccurate error calculation and infinite memory.  The implementation of the E-BMFLC filter is divided into two phases of enhancement; $\textit{Phase}$ 1) $\textit{Harmonic Model Enrichment:}$ To deal with the incorrect error calculation, and $\textit{Phase}$ 2) $\textit{Memory manipulation}$: Using a windowed memory instead of the conventional infinite memory. 

Instead of using the truncated model that only considers the frequency of the tremor $(m_{k-i}$), $[\omega_1,\omega_n]$ seen in the figure at the bottom, the complete frequency range of the hand motion ($m_k$),

\begin{equation}\label{eq:range}
[\omega_{min}, \omega_{max}]
\end{equation}

is considered. The frequency window is divided into a number of divisions 

\begin{equation}\label{eq:divisions}
n=\frac{\omega_{max}-\omega_{min}}{\Delta \omega}
\end{equation}

and $n$ is then the number of harmonics in for the Fourier combiner model of $m_k$. The following Fourier combiner is then used to model the complete hand motion $m_k$, where ${\omega_1} =\omega_{min}$ and ${\omega_n} =\omega_{max}$.



\begin{equation}
\hat{m}_k=\sum_{r=1}^{n}a_{rk}\sin(\omega_r k )+b_{rk}\cos(\omega_rk)
\end{equation}

Then $\textbf{x}_{k}$ and $\textbf{w}_{k}$ are defined 

\begin{align}
\textbf{x}_{k}&=
\begin{bmatrix}
[\sin(\omega_1k)&\sin(\omega_2k)&\cdots&\sin(\omega_nk)]^T \label{eq:45}\\
[\cos(\omega_1k)&\cos(\omega_2k)&\cdots&\cos(\omega_nk)]^T
\end{bmatrix} \\
\textbf{w}_{k}&=\begin{bmatrix}
[a_{1k}&a_{2k}&\cdots a_{nk}&]^T\label{eq:46}\\
[b_{1k}&b_{2k}&\cdots b_{nk}&]^T
\end{bmatrix} 
\end{align}

Using \eqref{eq:45} and \eqref{eq:46} the linear combiner can be written as

\begin{equation}
\hat{m}_k=\textbf{w}_{k}^T\textbf{x}_{k}
\end{equation}

The error between the signal of the complete motion of the hand $m_k $ and the estimated signal $\hat{m}_k$ is

\begin{equation}\label{key}
\varepsilon_{k}=m_k-\hat{m}_k
\end{equation}

and the recursive LMS algorithm used to update the weights of $\textbf{w}_k$

\begin{align}\label{eq:49}
\textbf{w}_{k+1}&=\rho \textbf{w}_{k}+2\mu\textbf{x}_{k}\varepsilon_k\\
\rho&=\sqrt[\delta]{\alpha}\\
\delta&=\frac{1}{\Delta T}T_p\label{eq:411}
\end{align}

In \eqref{eq:49}, $\rho$  defines the pole of discrete dynamics of the memory windowing for the filter in the Z-domain, considered to enhance the performance of the filter, especially for non-periodic signals. Decreasing $\rho$ increases the rate of forgetting past information. In \eqref{eq:411}, $\Delta T$ is the sampling time (in seconds), $T_p$ is the width of the considered memory window (in seconds), and $\alpha$ is the minimum amplification gain considered within the window.

After the estimated motion $\hat{m}_k$ has been modeled, the estimate of the voluntary motion $\hat{m}_{k-v}$ and  involuntary motion $\hat{m}_{k-i}$ can be extracted. To do this different band-limited windows of frequency can be considered to extract various frequency ranges. Using this technuiqe, the signal modeling and frequency truncation are decoupled, while in the formulation of BMFLC filter these two steps are fused. Two frequency ranges will be considered, $\left[\omega_{a},\omega_{b} \right]$ and $\left[\omega_{c},\omega_{d} \right]$, for voluntary and involuntary motion respectively. In order to extract $\hat{m}_{k-i}$ and $\hat{m}_{k-v}$, the calculated $\textbf{w}_k$ and $\textbf{x}_k$ should be truncated considering the frequency range of the voluntary and involuntary motion. The following definitions are made for this purpose

\begin{equation}
N_a=\dfrac{\omega_{a}-\omega_{min}}{\Delta\omega}, \hspace{8pt} N_b=\dfrac{\omega_{b}-\omega_{min}}{\Delta\omega}
\end{equation}

\begin{equation}
N_c=\dfrac{\omega_{c}-\omega_{min}}{\Delta\omega}, \hspace{8pt} N_d=\dfrac{\omega_{d}-\omega_{min}}{\Delta\omega}
\end{equation}

where $\omega_{min}$ is from \eqref{eq:range}, and $\Delta \omega$ is the same as in \eqref{eq:divisions}. The estimates for $\hat{m}_{k-i}$ and $\hat{m}_{k-v}$ can now be found with the following equations

\begin{equation}
\hat{m}_{k-v}=\textbf{w}_{k-v}^T\textbf{x}_{k-v}
\end{equation}


\begin{align}
\textbf{x}_{k-v}&=
\begin{bmatrix}
[\sin(\omega_{N_{a}}k)&\sin(\omega_{(N_{a}+1)}k)&\cdots&\sin(\omega_{N_{b}}k)]^T \\
[\cos(\omega_{N_{a}}k)&\cos(\omega_{(N_{a}+1)}k)&\cdots&\cos(\omega_{N_{b}}k)]^T
\end{bmatrix} \\
\textbf{w}_{k-v}&=\begin{bmatrix}
[a_{{N_{a}}k}&a_{({N_{a}+1})}k&\cdots & a_{{N_{b}}k}]^T\\
[b_{{N_{a}}k}&b_{(N_{a}+1)}k&\cdots & b_{{N_{b}}k}]^T
\end{bmatrix} 
\end{align}

\begin{equation}
\hat{m}_{k-i}=\textbf{w}_{k-i}^T\textbf{x}_{k-i}
\end{equation}

\begin{align}
\textbf{x}_{k-i}&=
\begin{bmatrix}
[\sin(\omega_{N_{c}}k)&\sin(\omega_{(N_{c}+1)}k)&\cdots&\sin(\omega_{N_{d}}k)]^T\\
[\cos(\omega_{N_{c}}k)&\cos(\omega_{(N_{c}+1)}k)&\cdots&\cos(\omega_{N_{d}}k)]^T
\end{bmatrix} \\
\textbf{w}_{k-i}&=\begin{bmatrix}
[a_{{N_{c}}k}&a_{({N_{c}+1})}k&\cdots& a_{{N_{d}}k}]^T\\
[b_{{N_{c}}k}&b_{(N_{c}+1)}k&\cdots& b_{{N_{d}}k}]^T
\end{bmatrix} 
\end{align}

![complete-band](fig/complete-band.png)